# Install

In [ ]:
!pip install -q -U bitsandbytes
!pip install -U peft==0.4.0
!pip install git+https://github.com/huggingface/transformers
!pip install -q -U accelerate
!pip install -q -U einops
!pip install -q -U safetensors
!pip install -q -U torch
!pip install -q -U xformers
!pip install -q -U datasets
!pip install -q -U trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 90.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 34.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 70.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 38.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 102.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 37.1 MB/s eta 0:00:00
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-iq3mqe_p
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-iq3mqe_p
  Resolved https://github.com/huggingface/transformers to commit 9beb2737d758160e845b66742a0c01201e38007f
  Installing build dependencies ..

In [ ]:
!pip install transformers[sentencepiece]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.8 MB/s eta 0:00:00


# Import

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline, TextStreamer, TrainingArguments
import transformers
import torch
from torch.utils.data import DataLoader, Dataset
import torch
from transformers import AutoTokenizer, pipeline

# Load Quantized Model

In [ ]:
model_id = "FelixChao/Mistral-7b-Chem-TW" # sharded model by vilsonrodrigues
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,# 嵌套量化，每个参数可以多节省0.4位
    bnb_4bit_quant_type="nf4",#  NF4（normalized float）或纯FP4量化 博客说推荐NF4
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained("FelixChao/Mistral-7b-TW-base",trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(model_id,device_map={"": 0})

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

In [ ]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
model

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )

In [ ]:
from peft import LoraConfig, get_peft_model

target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
        "lm_head",
        "embed_tokens",
       ]

config = LoraConfig(
    r=64,
    lora_alpha=16,
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=target_modules,
)


model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 172392448 || all params: 3924463616 || trainable%: 4.392764588188757


In [ ]:
print_trainable_parameters(model)

trainable params: 172392448 || all params: 3924463616 || trainable%: 4.392764588188757


# Prepare Data

In [ ]:
from datasets import load_dataset

dataset = load_dataset("FelixChao/chemistry_zh-tw",split='train')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/40000 [00:00<?, ? examples/s]

In [ ]:
print(dataset[6])

{'chem_text': '### 指令:\n“確定以下有機化合物的 IUPAC 名稱：CH3CH2CH2COOH”\n\n### 回答:\n有機化合物CH3CH2CH2COOH的IUPAC名稱為丁酸。'}


In [ ]:
def format_dolly(sample):
    instruction = f"### 指令:\n{sample['instruction']}"
    context = f"### 文章:\n{sample['context']}" if len(sample["context"]) > 0 else None
    response = f"### 回答:\n{sample['response']}"
    # join all the parts together
    prompt = "\n\n".join([i for i in [instruction, context, response] if i is not None])
    return prompt

In [ ]:
from random import randrange

print(format_dolly(dataset[randrange(len(dataset))]))

KeyError: ignored

In [ ]:
# template dataset to add prompt to each sample
from random import randint
def template_dataset(sample):
    sample["text"] = f"{format_dolly(sample)}{tokenizer.eos_token}"
    return sample

dataset = dataset.map(template_dataset, remove_columns=list(dataset.features))

print(dataset[randint(0, len(dataset))]["text"])

### 指令:
從這個摘要中，列出布萊恩·克里斯汀在他的書《對齊問題》的三個章節中涵蓋的人工智慧系統表現出與人類價值觀不一致的三種方式。

### 文章:
這本書分為三個部分：預言、代理和規範性。每個部分都涵蓋了研究人員和工程師在人工智慧與人類價值觀對齊方面面臨的不同挑戰。

預言
在第一部分中，克里斯蒂安交織了人工智慧研究的歷史，特別是人工神經網路（如感知器和AlexNet）的機器學習方法，以及人工智慧系統可能存在意外行為的例子。他講述了記者朱莉婭·安格溫的故事，她對預測犯罪被告再犯率的工具COMPAS演演算法進行了調查，引起了對其準確性和對某些人群的偏見的廣泛批評。人工智慧的主要對齊挑戰之一是其黑匣子性質（輸入和輸出可識別，但中間的轉換過程不確定）。缺乏透明度使得很難知道系統在哪些方面做得好，在哪些方面做得不好。

代理
在第二部分中，克里斯蒂安同樣交織了獎勵心理學研究的歷史，如行為主義和多巴胺，與強化學習的電腦科學，其中人工智慧系統需要在價值函式（「期望獲得的獎勵或懲罰」）面前制定策略（「該做什麼」）。他稱DeepMind AlphaGo和AlphaZero系統為「自動課程設計中可能是最令人印象深刻的成就」。他還強調了好奇心的重要性，其中強化學習者具有內在動機去探索他們的環境，而不僅僅是尋求外部獎勵。

規範性
第三部分涵蓋了透過模仿人類或機器行為來訓練人工智慧，以及哲學上的辯論，如可能主義和實在主義之間的辯論，這意味著對於人工智慧系統有不同的理想行為。特別重要的是逆強化學習，這是一種廣泛的方法，用於讓機器學習人類或其他代理的目標函式。克里斯蒂安討論了有效利他主義和存在風險等規範性挑戰，包括哲學家託比·奧德和威廉·麥卡斯基爾克正在嘗試制定人類和機器策略，以儘可能有效地解決對齊問題。`

### 回答:
在第一部分“預言”中，作者討論了預測模型做出的決策難以檢查，並且可能變得具有歧視性或不公平性。

在第二部分中，關於強化學習系統的代理，儘管取得了一些顯著的成就，但幾乎完全由其目標函式引導，而不是由探索世界的好奇心引導。

在第三部分“規範性”中，逆強化學習技術面臨著找出人類目標函式的挑戰。</s>


# Example Before Fine Tuning

In [ ]:
def generate(index):

  example_text = "### 指令: What is the IUPAC name of the following organic compound: CH3CH2CH2COOH?\n\n### 回答:"

  print("Question:")
  print(example_text)

  streamer = TextStreamer(tokenizer=tokenizer,skip_prompt=True,skip_special_tokens=True)
  encoding = tokenizer(example_text, return_tensors="pt").to("cuda:0")
  output = model.generate(input_ids=encoding.input_ids, streamer = streamer, attention_mask=encoding.attention_mask, max_new_tokens=500, eos_token_id=tokenizer.eos_token_id, temperature=0.00001, do_sample=True)
  predict = tokenizer.decode(output[0], skip_special_tokens=True)
  # print("Answer:")
  # print(predict)

  print()

In [ ]:
generate(10)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Question:
### 指令: What is the IUPAC name of the following organic compound: CH3CH2CH2COOH?

### 回答:

給定有機化合物的 IUPAC 名稱是丁酸。它是一種四碳酸，具體是第二個碳原子上連線有兩個碳原子和兩個氫原子的直鏈酸。 IUPAC 命名採用了系統和遵循特定規則的方法，例如基組和官能基的排列。在這種情況下，IUPAC 名稱是丁酸，因為它是從碳鏈中開始，並具有酸官能基。另一方面，如果化合物以其官能基命名，則它將稱為乙酸乙酯，因為它是乙酸與乙醯基的酯。乙醯基。兩者代表

KeyboardInterrupt: ignored

In [ ]:
from trl import SFTTrainer

In [ ]:
training_arguments = TrainingArguments(
        num_train_epochs=3,
        per_device_train_batch_size=16,
        gradient_accumulation_steps=4,
        warmup_ratio=0.05,
        max_steps=200,
        learning_rate=2e-4,
        bf16=True,
        fp16=False,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_32bit",
        lr_scheduler_type='cosine',
    )

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    dataset_text_field="chem_text",
    max_seq_length=1024,
    tokenizer=tokenizer,
    args=training_arguments,
)

Map:   0%|          | 0/40000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:214: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


In [ ]:
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
1,0.334200
2,0.345900
3,0.311600
4,0.371600
5,0.324600
6,0.336900
7,0.351400
8,0.338700
9,0.323500
10,0.356600


TrainOutput(global_step=200, training_loss=0.32432217195630075, metrics={'train_runtime': 6956.5472, 'train_samples_per_second': 1.84, 'train_steps_per_second': 0.029, 'total_flos': 5.7274368015610675e+17, 'train_loss': 0.32432217195630075, 'epoch': 0.32})

# Example After Fine Tuning

In [ ]:
model.config.use_cache = True
model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(
          32000, 4096
          (lora_dropout): ModuleDict(
            (default): Dropout(p=0.1, inplace=False)
          )
          (lora_A): ModuleDict()
          (lora_B): ModuleDict()
          (lora_embedding_A): ParameterDict(  (default): Parameter containing: [torch.cuda.FloatTensor of size 64x32000 (cuda:0)])
          (lora_embedding_B): ParameterDict(  (default): Parameter containing: [torch.cuda.FloatTensor of size 4096x64 (cuda:0)])
        )
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralAttention(
              (q_proj): Linear4bit(
                in_features=4096, out_features=4096, bias=False
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                

In [ ]:
generate(15)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Question:
### 指令:
奈米粒子的表面修飾如何影響其在各種化學反應中的穩定性和反應性？


### 回答:
奈米粒子的表面修飾在決定其在各種化學反應中的穩定性和反應性方面起著至

KeyboardInterrupt: ignored

In [ ]:
from peft import PeftModel, PeftConfig
config = PeftConfig.from_pretrained("FelixChao/Mistral-7b-Chem-TW-Lora-v2")
model = PeftModel.from_pretrained(model, "FelixChao/Mistral-7b-Chem-TW-Lora-v2")

In [ ]:
model = model.merge_and_unload()

In [ ]:
import huggingface_hub
huggingface_hub.notebook_login()

In [ ]:
model.push_to_hub("Mistral-7b-Chem-TW-v2")

model-00003-of-00006.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00005-of-00006.safetensors:   0%|          | 0.00/4.83G [00:00<?, ?B/s]

model-00006-of-00006.safetensors:   0%|          | 0.00/4.25G [00:00<?, ?B/s]

Upload 6 LFS files:   0%|          | 0/6 [00:00<?, ?it/s]

model-00001-of-00006.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00006.safetensors:   0%|          | 0.00/4.90G [00:00<?, ?B/s]

model-00004-of-00006.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/FelixChao/Mistral-7b-Chem-TW-v2/commit/9552a809b14f88cdd29e5e5554ff2950c0adf877', commit_message='Upload MistralForCausalLM', commit_description='', oid='9552a809b14f88cdd29e5e5554ff2950c0adf877', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
tokenizer.push_to_hub("Mistral-7b-Chem-TW-v2")

CommitInfo(commit_url='https://huggingface.co/FelixChao/Mistral-7b-Chem-TW-v2/commit/6df2a7f1ebccb3470f1763660745eb3ba36a3e36', commit_message='Upload tokenizer', commit_description='', oid='6df2a7f1ebccb3470f1763660745eb3ba36a3e36', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
prediction2 = get_response_set(model)

In [ ]:
prediction2

['To determine the maximum amount of sodium chloride (NaCl) that can dissolve in 100 mL of water at 25°C, we need to consider the solubility of NaCl in water.\n\nThe solubility of NaCl in water is approximately 35.2 g/100 mL at 25°C. This means that 35.2 g of NaCl can dissolve in 100 mL of water at 25°C.\n\nSo, the maximum amount of sodium chloride that can dissolve in 100 mL of water at 25°C is 35.2 g.\n\nPlease note that this is an approximation, and the actual amount of NaCl that can dissolve in 100 mL of water may vary slightly depending on the specific conditions of the experiment.',
 'The balanced chemical equation for the neutralization reaction between sulfuric acid (H₂SO₄) and sodium hydroxide (NaOH) is:\n\nH₂SO₄(aq) + 2NaOH(aq) → Na₂SO₄(aq) + 2H₂O(l)\n\nIn this reaction, one mole of sulfuric acid reacts with two moles of sodium hydroxide to produce one mole of sodium sulfate and two moles of water.',
 'To calculate the standard enthalpy change (ΔH°) for the reaction, we can u

In [ ]:
results2 = rouge.compute(predictions=prediction2,references=references,use_stemmer=True)

In [ ]:
results2

{'rouge1': 0.5521110495119378,
 'rouge2': 0.3381439439218584,
 'rougeL': 0.3784680506809217,
 'rougeLsum': 0.5111232522267442}

In [ ]:
bleu = evaluate.load('bleu')
results_bleu = bleu.compute(predictions=predictions, references=references)
results_bleu2 = bleu.compute(predictions=prediction2, references=references)

print(results_bleu)
print(results_bleu2)

{'bleu': 0.06389320763725576, 'precisions': [0.6257722859664607, 0.2839396628216504, 0.15611061552185548, 0.10224215246636771], 'brevity_penalty': 0.27687165053192225, 'length_ratio': 0.437789799072643, 'translation_length': 1133, 'reference_length': 2588}
{'bleu': 0.23996042784684243, 'precisions': [0.48559296329996965, 0.282628536659568, 0.18614586512053707, 0.1297826752372207], 'brevity_penalty': 1.0, 'length_ratio': 1.2739567233384854, 'translation_length': 3297, 'reference_length': 2588}


In [ ]:
toxicity = evaluate.load("toxicity")
results_tox = toxicity.compute(predictions=predictions, aggregation="ratio")
results_tox2 = toxicity.compute(predictions=prediction2, aggregation="ratio")
print(results_tox)
print(results_tox2)

{'toxicity_ratio': 0.0}
{'toxicity_ratio': 0.0}


In [ ]:
df.insert(loc = 2, column="vicuna7b_model_response", value=prediction2)

In [ ]:
df

,quetsion,response,vicuna7b_model_response,answer
0,What is the maximum amount of sodium chloride ...,,To determine the maximum amount of sodium chlo...,The solubility of sodium chloride (NaCl) in wa...
1,What is the balanced chemical equation for the...,The balanced chemical equation for the neutral...,The balanced chemical equation for the neutral...,The balanced chemical equation for the neutral...
2,Calculate the standard enthalpy change and ent...,,To calculate the standard enthalpy change (ΔH°...,To calculate the standard enthalpy change (ΔH°...
3,"""How does the activity of a catalyst influence...",\nThe activity of a catalyst can influence the...,The activity of a catalyst plays a crucial rol...,The activity of a catalyst influences the reac...
4,What is the most effective way to inhibit the ...,\nMolecular docking is a computational method ...,The most effective way to inhibit the interact...,The most effective way to inhibit the interact...
5,How can we optimize the yield of polystyrene s...,\nPolystyrene is a synthetic polymer that is w...,To optimize the yield of polystyrene synthesis...,To optimize the yield of polystyrene synthesis...
6,What is the standard enthalpy of formation of ...,,To find the standard enthalpy of formation of ...,The given information is not sufficient to dir...
7,Calculate the standard electrode potential for...,,To calculate the standard electrode potential ...,To calculate the standard electrode potential ...
8,"""How can the pharmacology of natural products ...",\nThe pharmacology of natural products is a co...,The pharmacology of natural products can be in...,The pharmacology of natural products can be in...
9,What is the nuclear decay mode of a carbon-14 ...,The nuclear decay mode of a carbon-14 atom is ...,Carbon-14 (C14) is a radioactive isotope of ca...,Carbon-14 undergoes beta-minus decay (β- decay...


In [ ]:
c1 = "rouge1"
c2 = "rouge2"
cL = "rougeL"
cLsum = "rougeLsum"
df_score = pd.DataFrame({
    c1 : [results[c1], results2[c1], results3[c1]],
    c2 : [results[c2], results2[c2], results3[c2]],
    cL : [results[cL], results2[cL], results3[cL]],
    cLsum : [results[cLsum], results2[cLsum], results3[cLsum]],
})
df_score.index = ["falcon7b" , "ESG_model", "vicuna7b"]
df_score

,rouge1,rouge2,rougeL,rougeLsum
falcon7b,0.466976,0.145024,0.273879,0.294289
ESG_model,0.525273,0.203351,0.325743,0.354960
vicuna7b,0.457632,0.168178,0.277941,0.301291


# Merge Adapters and Save Model to Hub

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
model.save_pretrained("Mistral-7b-Lora-tw", push_to_hub=True) # save LORA adapters
model.push_to_hub("Mistral-7b-Lora-tw")

adapter_model.bin:   0%|          | 0.00/680M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/FelixChao/Mistral-7b-Lora-tw/commit/5c9fef572c33cb0e19a90771720d7fe0ef8ba06d', commit_message='Upload model', commit_description='', oid='5c9fef572c33cb0e19a90771720d7fe0ef8ba06d', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
model.print_trainable_parameters()

trainable params: 8,388,608 || all params: 6,746,804,224 || trainable%: 0.12433454005023165


In [ ]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(
          32000, 4096
          (lora_dropout): ModuleDict(
            (default): Dropout(p=0.1, inplace=False)
          )
          (lora_A): ModuleDict()
          (lora_B): ModuleDict()
          (lora_embedding_A): ParameterDict(  (default): Parameter containing: [torch.cuda.FloatTensor of size 64x32000 (cuda:0)])
          (lora_embedding_B): ParameterDict(  (default): Parameter containing: [torch.cuda.FloatTensor of size 4096x64 (cuda:0)])
        )
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralAttention(
              (q_proj): Linear(
                in_features=4096, out_features=4096, bias=False
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (d

In [ ]:
!pip uninstall peft

Found existing installation: peft 0.5.0
Uninstalling peft-0.5.0:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/peft-0.5.0.dist-info/*
    /usr/local/lib/python3.10/dist-packages/peft/*
Proceed (Y/n)? y
  Successfully uninstalled peft-0.5.0


In [ ]:
!pip install peft==0.2.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 1.3 MB/s eta 0:00:00


In [ ]:
model = model.merge_and_upload()

NameError: ignored

In [ ]:
from langchain import PromptTemplate
from langchain.chains import ConversationChain
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.llms import HuggingFacePipeline
from langchain.schema import BaseOutputParser

In [ ]:
model.generation_config

GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 1,
  "do_sample": true,
  "eos_token_id": 2,
  "max_new_tokens": 1000,
  "pad_token_id": 2,
  "repetition_penalty": 1.7,
  "temperature": 1e-06,
  "top_k": 0,
  "transformers_version": "4.32.0.dev0",
  "use_cache": false
}

In [ ]:
generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,
    task="text-generation",
    generation_config=generation_config,
)

llm = HuggingFacePipeline(pipeline=generation_pipeline)

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PLBartForCausalLM', 'ProphetNetForCausalLM', 'QDQBertLMHeadModel', 'ReformerModelWithLMHead',

In [ ]:
prompt = "What volume of 0.2 M NaOH is required to completely neutralize 25 mL of 0.1 M HCl?"
res = llm(prompt)
print(res)


To solve this problem, we will use the concept called "Molarity and Concentration." Firstly, let's find out how many moles (moles) are present in both solutions:


In [ ]:
#use adapter 1
model.set_adapter("adapter_1")
output_1 = model.generate(**inputs)
print(tokenizer.decode(output_1[0],skip_special_tokens=True))

#use adapter 2
model.set_adapter("adapter_2")
output_2 = model.generate(**inputs)
print(tokenizer.decode(output_2[0],skip_special_tokens=True))

In [ ]:
dpo_trainer = DPOTrainer(
    model,                 # base model from SFT pipeline
    model_ref,             # typically a copy of the SFT trained base model
    beta=0.1,              # temperature hyperparameter of DPO
    train_dataset=dataset, # dataset prepared above
    tokenizer=tokenizer,   # tokenizer
    args=training_args,    # training arguments e.g. batch size, lr, etc.
)